In [3]:
import pandas as pd 
df = pd.read_csv("/kaggle/input/medicaltranscriptions/mtsamples.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         4999 non-null   int64 
 1   description        4999 non-null   object
 2   medical_specialty  4999 non-null   object
 3   sample_name        4999 non-null   object
 4   transcription      4966 non-null   object
 5   keywords           3931 non-null   object
dtypes: int64(1), object(5)
memory usage: 234.5+ KB


In [ ]:
# =====================================================
# COMPLETE MEDICAL RAG SYSTEM - PRODUCTION READY
# =====================================================

# Step 1: Install Required Packages
print("Installing required packages...")
!pip install -q langchain sentence-transformers faiss-cpu transformers torch gradio pandas numpy

# Step 2: Import Libraries
import pandas as pd
import numpy as np
import pickle
import faiss
import gradio as gr
import warnings
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!\n")

# =====================================================
# STEP 1: DATA LOADING AND PREPROCESSING
# =====================================================

class MedicalDataLoader:
    """Loads and preprocesses medical transcription data"""
    
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = None
        
    def load_data(self) -> pd.DataFrame:
        """Load CSV data with validation"""
        print("📂 Loading medical dataset...")
        try:
            self.df = pd.read_csv(self.csv_path)
            print(f"✅ Loaded {len(self.df)} records")
            print(f"   Columns: {list(self.df.columns)}")
            
            # Data validation
            self._validate_data()
            self._clean_data()
            
            return self.df
            
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return None
    
    def _validate_data(self):
        """Validate required columns exist"""
        required_cols = ['transcription', 'medical_specialty']
        missing = [col for col in required_cols if col not in self.df.columns]
        
        if missing:
            raise ValueError(f"Missing required columns: {missing}")
        
        print(f"✅ Data validation passed")
    
    def _clean_data(self):
        """Clean and preprocess data"""
        # Handle missing values
        self.df['transcription'] = self.df['transcription'].fillna('')
        self.df['medical_specialty'] = self.df['medical_specialty'].fillna('Unknown')
        self.df['keywords'] = self.df['keywords'].fillna('')
        self.df['description'] = self.df['description'].fillna('')
        self.df['sample_name'] = self.df['sample_name'].fillna('')
        
        # Filter out empty transcriptions
        initial_count = len(self.df)
        self.df = self.df[self.df['transcription'].str.len() > 100]
        removed = initial_count - len(self.df)
        
        print(f"✅ Data cleaned: removed {removed} short transcriptions")

# =====================================================
# STEP 2: SMART CHUNKING STRATEGY
# =====================================================

class MedicalChunker:
    """Medical-optimized text chunking with context preservation"""
    
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )
    
    def create_chunks(self, df: pd.DataFrame) -> Tuple[List[str], List[Dict]]:
        """Create chunks with rich metadata"""
        print("\n📝 Creating medical text chunks...")
        
        all_chunks = []
        all_metadata = []
        
        for idx, row in df.iterrows():
            # Get text and metadata
            text = str(row.get('transcription', ''))
            specialty = str(row.get('medical_specialty', 'Unknown'))
            sample_name = str(row.get('sample_name', 'Unknown'))
            keywords = str(row.get('keywords', ''))
            description = str(row.get('description', ''))
            
            # Skip very short texts
            if len(text) < 100:
                continue
            
            # Create chunks
            chunks = self.text_splitter.split_text(text)
            
            # Add each chunk with metadata
            for chunk_idx, chunk in enumerate(chunks):
                all_chunks.append(chunk)
                all_metadata.append({
                    'medical_specialty': specialty,
                    'sample_name': sample_name,
                    'keywords': keywords,
                    'description': description,
                    'doc_index': int(idx),
                    'chunk_index': chunk_idx,
                    'total_chunks': len(chunks),
                    'chunk_length': len(chunk)
                })
        
        print(f"✅ Created {len(all_chunks)} chunks from {len(df)} documents")
        print(f"   Average chunk size: {np.mean([len(c) for c in all_chunks]):.0f} characters")
        
        return all_chunks, all_metadata

# =====================================================
# STEP 3: VECTOR STORE CREATION
# =====================================================

class MedicalVectorStore:
    """Creates and manages FAISS vector store with medical embeddings"""
    
    def __init__(self, model_name: str = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"):
        print(f"\n🤖 Loading embedding model: {model_name}")
        try:
            self.embedder = SentenceTransformer(model_name)
            print("✅ Medical embedding model loaded")
        except:
            print("⚠️  Medical model not available, using general model...")
            self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        
        self.index = None
        self.documents = []
        self.metadatas = []
    
    def create_index(self, chunks: List[str], metadatas: List[Dict]) -> faiss.Index:
        """Create FAISS index with embeddings"""
        print("\n🔢 Creating embeddings and FAISS index...")
        
        self.documents = chunks
        self.metadatas = metadatas
        
        # Create embeddings in batches
        batch_size = 32
        all_embeddings = []
        
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            embeddings = self.embedder.encode(
                batch,
                convert_to_numpy=True,
                show_progress_bar=True,
                normalize_embeddings=True
            )
            all_embeddings.append(embeddings)
        
        # Combine all embeddings
        embeddings_array = np.vstack(all_embeddings).astype('float32')
        
        # Create FAISS index
        dimension = embeddings_array.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner product for normalized vectors
        self.index.add(embeddings_array)
        
        print(f"✅ FAISS index created with {self.index.ntotal} vectors")
        print(f"   Embedding dimension: {dimension}")
        
        return self.index
    
    def save(self, path: str = "medical_faiss_index"):
        """Save vector store to disk"""
        print(f"\n💾 Saving vector store to '{path}'...")
        
        import os
        os.makedirs(path, exist_ok=True)
        
        # Save FAISS index
        faiss.write_index(self.index, f"{path}/index.faiss")
        
        # Save documents and metadata
        with open(f"{path}/index.pkl", "wb") as f:
            pickle.dump({
                'documents': self.documents,
                'metadatas': self.metadatas
            }, f)
        
        print("✅ Vector store saved successfully!")
        print(f"   - {path}/index.faiss ({os.path.getsize(f'{path}/index.faiss') / 1024 / 1024:.2f} MB)")
        print(f"   - {path}/index.pkl ({os.path.getsize(f'{path}/index.pkl') / 1024 / 1024:.2f} MB)")
    
    def load(self, path: str = "medical_faiss_index"):
        """Load vector store from disk"""
        print(f"\n📂 Loading vector store from '{path}'...")
        
        try:
            # Load FAISS index
            self.index = faiss.read_index(f"{path}/index.faiss")
            print(f"✅ FAISS index loaded: {self.index.ntotal} vectors")
            
            # Load documents and metadata
            with open(f"{path}/index.pkl", "rb") as f:
                data = pickle.load(f)
                self.documents = data['documents']
                self.metadatas = data['metadatas']
            
            print(f"✅ Documents loaded: {len(self.documents)} chunks")
            return True
            
        except Exception as e:
            print(f"❌ Error loading vector store: {e}")
            return False

# =====================================================
# STEP 4: MEDICAL LLM FOR ANSWER GENERATION
# =====================================================

class MedicalLLM:
    """Medical language model for answer generation"""
    
    def __init__(self):
        print("\n🧠 Loading medical language model...")
        
        # Try medical models, fallback to general
        try:
            model_name = "google/flan-t5-base"  # Good balance of quality and speed
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model.to(self.device)
            print(f"✅ Model loaded: {model_name} on {self.device}")
        except Exception as e:
            print(f"⚠️  Could not load model: {e}")
            self.model = None
    
    def generate_answer(self, question: str, context: str, max_length: int = 300) -> str:
        """Generate answer from question and context"""
        
        if self.model is None:
            return self._fallback_answer(context)
        
        # Create prompt
        prompt = f"""Based on the following medical information, provide a clear and accurate answer to the question.

Medical Context:
{context}

Question: {question}

Answer:"""
        
        # Tokenize
        inputs = self.tokenizer(
            prompt,
            max_length=1024,
            truncation=True,
            return_tensors="pt"
        ).to(self.device)
        
        # Generate
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=max_length,
                num_beams=4,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                no_repeat_ngram_size=3
            )
        
        # Decode
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return answer.strip()
    
    def _fallback_answer(self, context: str) -> str:
        """Fallback when model not available"""
        return f"Based on the retrieved information:\n\n{context[:500]}...\n\n(Note: Full LLM generation unavailable)"

# =====================================================
# STEP 5: COMPLETE RAG SYSTEM
# =====================================================

class MedicalRAGSystem:
    """Complete RAG system with retrieval and generation"""
    
    def __init__(self, index_path: str = "medical_faiss_index"):
        print("\n" + "="*60)
        print("   INITIALIZING MEDICAL RAG SYSTEM")
        print("="*60)
        
        # Load vector store
        self.vector_store = MedicalVectorStore()
        self.vector_store.load(index_path)
        
        # Load LLM
        self.llm = MedicalLLM()
        
        print("\n✅ Medical RAG System ready!")
        print("="*60 + "\n")
    
    def retrieve(self, query: str, k: int = 5, specialty_filter: str = None) -> List[Dict]:
        """Retrieve relevant documents"""
        
        # Encode query
        query_embedding = self.vector_store.embedder.encode(
            [query],
            convert_to_numpy=True,
            normalize_embeddings=True
        ).astype('float32')
        
        # Search
        k_search = k * 3 if specialty_filter else k
        distances, indices = self.vector_store.index.search(query_embedding, k_search)
        
        # Prepare results
        results = []
        for dist, idx in zip(distances[0], indices[0]):
            if idx == -1 or idx >= len(self.vector_store.documents):
                continue
            
            metadata = self.vector_store.metadatas[idx]
            
            # Apply specialty filter
            if specialty_filter and metadata['medical_specialty'] != specialty_filter:
                continue
            
            results.append({
                'text': self.vector_store.documents[idx],
                'metadata': metadata,
                'score': float(dist)
            })
            
            if len(results) >= k:
                break
        
        return results
    
    def ask(self, question: str, k: int = 5, specialty_filter: str = None) -> Dict:
        """Complete RAG pipeline: retrieve and generate"""
        
        # Add medical disclaimer to question
        if not any(word in question.lower() for word in ['how', 'what', 'when', 'where', 'why', 'explain']):
            question = f"Explain: {question}"
        
        # Retrieve relevant chunks
        retrieved_docs = self.retrieve(question, k=k, specialty_filter=specialty_filter)
        
        if not retrieved_docs:
            return {
                'answer': "I couldn't find relevant medical information for your question. Please try rephrasing or ask about a different topic.",
                'sources': [],
                'specialties': []
            }
        
        # Combine context
        context = "\n\n".join([
            f"[Source {i+1} - {doc['metadata']['medical_specialty']}]\n{doc['text']}"
            for i, doc in enumerate(retrieved_docs[:3])
        ])
        
        # Generate answer
        answer = self.llm.generate_answer(question, context)
        
        # Extract specialties
        specialties = list(set([doc['metadata']['medical_specialty'] for doc in retrieved_docs]))
        
        # Format sources
        sources = []
        for i, doc in enumerate(retrieved_docs):
            sources.append({
                'rank': i + 1,
                'specialty': doc['metadata']['medical_specialty'],
                'sample': doc['metadata']['sample_name'],
                'text_preview': doc['text'][:200] + "...",
                'relevance_score': f"{doc['score']:.3f}"
            })
        
        return {
            'answer': answer,
            'sources': sources,
            'specialties': specialties,
            'disclaimer': "⚠️ This is AI-generated information. Always consult healthcare professionals for medical advice."
        }

# =====================================================
# STEP 6: BUILD VECTOR STORE (RUN ONCE)
# =====================================================

def build_vector_store(csv_path: str = '/content/mtsamples.csv'):
    """Complete pipeline to build vector store"""
    
    print("\n" + "="*60)
    print("   BUILDING MEDICAL VECTOR STORE")
    print("="*60 + "\n")
    
    # Load data
    loader = MedicalDataLoader(csv_path)
    df = loader.load_data()
    
    if df is None:
        print("❌ Failed to load data. Please check the file path.")
        return False
    
    # Create chunks
    chunker = MedicalChunker(chunk_size=1000, chunk_overlap=200)
    chunks, metadatas = chunker.create_chunks(df)
    
    # Create vector store
    vector_store = MedicalVectorStore()
    vector_store.create_index(chunks, metadatas)
    
    # Save
    vector_store.save("medical_faiss_index")
    
    print("\n" + "="*60)
    print("   ✅ VECTOR STORE BUILD COMPLETE!")
    print("="*60 + "\n")
    
    return True

# =====================================================
# STEP 7: GRADIO UI
# =====================================================

def create_ui(rag_system: MedicalRAGSystem):
    """Create Gradio interface"""
    
    def answer_question(question, specialty_filter, num_sources):
        if not question.strip():
            return "Please enter a medical question.", ""
        
        # Get answer
        result = rag_system.ask(
            question,
            k=int(num_sources),
            specialty_filter=specialty_filter if specialty_filter != "All Specialties" else None
        )
        
        # Format answer
        answer_text = f"## 💬 Answer\n\n{result['answer']}\n\n"
        answer_text += f"{result['disclaimer']}\n\n"
        answer_text += f"**Related Specialties:** {', '.join(result['specialties'][:3])}\n\n"
        
        # Format sources
        sources_text = "## 📚 Sources\n\n"
        for source in result['sources']:
            sources_text += f"**{source['rank']}. {source['specialty']}** (Relevance: {source['relevance_score']})\n"
            sources_text += f"*Sample: {source['sample']}*\n"
            sources_text += f"{source['text_preview']}\n\n"
        
        return answer_text, sources_text
    
    # Get available specialties
    specialties = ["All Specialties"] + sorted(list(set([
        m['medical_specialty'] for m in rag_system.vector_store.metadatas
    ])))
    
    # Create interface
    with gr.Blocks(title="Medical RAG Assistant", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🏥 Medical RAG Assistant")
        gr.Markdown("Ask medical questions and get AI-powered answers with sources from medical transcriptions.")
        
        with gr.Row():
            with gr.Column(scale=2):
                question_input = gr.Textbox(
                    label="Enter Medical Question",
                    placeholder="e.g., What are the symptoms of hypertension?",
                    lines=3
                )
                
                with gr.Row():
                    specialty_dropdown = gr.Dropdown(
                        choices=specialties,
                        value="All Specialties",
                        label="Filter by Specialty"
                    )
                    num_sources_slider = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=5,
                        step=1,
                        label="Number of Sources"
                    )
                
                submit_btn = gr.Button("🔍 Search", variant="primary")
            
            with gr.Column(scale=3):
                answer_output = gr.Markdown(label="Answer")
                sources_output = gr.Markdown(label="Sources")
        
        # Examples
        gr.Examples(
            examples=[
                ["What are the common symptoms of diabetes?", "All Specialties", 5],
                ["How is hypertension diagnosed?", "Cardiovascular / Pulmonary", 5],
                ["Explain the procedure for knee arthroscopy", "Orthopedic", 5],
                ["What are treatment options for asthma?", "All Specialties", 5],
            ],
            inputs=[question_input, specialty_dropdown, num_sources_slider]
        )
        
        # Connect
        submit_btn.click(
            fn=answer_question,
            inputs=[question_input, specialty_dropdown, num_sources_slider],
            outputs=[answer_output, sources_output]
        )
    
    return demo

# =====================================================
# MAIN EXECUTION
# =====================================================

if __name__ == "__main__":
    import os
    
    # Step 1: Build vector store (run once)
    if not os.path.exists("medical_faiss_index/index.faiss"):
        print("🔨 Building vector store for the first time...")
        success = build_vector_store('/kaggle/input/medicaltranscriptions/mtsamples.csv')
        if not success:
            print("❌ Failed to build vector store. Exiting.")
            exit(1)
    else:
        print("✅ Vector store already exists, skipping build...")
    
    # Step 2: Initialize RAG system
    rag_system = MedicalRAGSystem("medical_faiss_index")
    
    # Step 3: Launch UI
    demo = create_ui(rag_system)
    demo.launch(share=True, debug=True)
    
    print("\n" + "="*60)
    print("   🚀 MEDICAL RAG SYSTEM RUNNING!")
    print("="*60)

Installing required packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0

2025-12-05 18:44:36.728540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764960276.953606      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764960277.022171      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model loaded: google/flan-t5-base on cuda

✅ Medical RAG System ready!

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a2bdba3c4388166b7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
